<a href="https://colab.research.google.com/github/kok554/computervision/blob/main/MLP%EA%B5%AC%ED%98%84%EA%B3%B5%EB%B6%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 필요한 라이브러리 가져오기
import torch  # PyTorch 라이브러리 (주로 텐서 연산 및 모델 학습)
import torch.nn as nn  # 신경망 모델을 구축하기 위한 모듈
import torch.nn.functional as F  # 신경망의 함수들 (예: 활성화 함수, 손실 함수 등)
import torch.optim as optim  # 최적화 알고리즘을 위한 모듈
from torchvision import datasets, transforms  # torchvision 라이브러리 (MNIST 데이터셋 및 이미지 변환)
from torch.utils.data import DataLoader  # 데이터를 배치로 로드할 수 있는 DataLoader

# 이미지 변환을 위한 변환 파이프라인 정의
transform = transforms.Compose([
    transforms.ToTensor(),  # 이미지를 PyTorch 텐서로 변환 (값 범위: [0, 1])
    transforms.Normalize((0.1307,), (0.3081,))  # MNIST 데이터셋의 평균 및 표준편차를 기준으로 정규화
])

# MNIST 훈련 데이터셋 로드
train_dataset = datasets.MNIST(
    './data',  # 데이터셋을 저장할 디렉터리 경로
    train=True,  # 훈련 데이터셋
    download=True,  # 데이터셋이 없다면 자동으로 다운로드
    transform=transform  # 앞에서 정의한 변환(transform)을 적용
)

# MNIST 테스트 데이터셋 로드
test_dataset = datasets.MNIST(
    './data',  # 데이터셋을 저장할 디렉터리 경로
    train=False,  # 테스트 데이터셋
    transform=transform  # 앞에서 정의한 변환(transform)을 적용
)

# 훈련 데이터셋을 배치 크기 64로 DataLoader로 로드
train_loader = DataLoader(
    train_dataset,  # 훈련 데이터셋
    batch_size=64,  # 배치 크기 (한 번에 64개의 샘플을 로드)
    shuffle=True  # 데이터를 섞어서 로드 (매 epoch마다 데이터가 섞임)
)

# 테스트 데이터셋을 배치 크기 1000으로 DataLoader로 로드
test_loader = DataLoader(
    test_dataset,  # 테스트 데이터셋
    batch_size=1000,  # 배치 크기 (한 번에 1000개의 샘플을 로드)
    shuffle=False  # 테스트 데이터는 순차적으로 로드 (섞지 않음)
)


Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9.91M/9.91M [00:00<00:00, 17.5MB/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28.9k/28.9k [00:00<00:00, 494kB/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1.65M/1.65M [00:00<00:00, 4.54MB/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4.54k/4.54k [00:00<00:00, 7.65MB/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [2]:
# MLP 클래스 정의
class MLP(nn.Module):
  def __init__(self):
    """
    클래스 초기화 메서드.
    MLP 모델의 레이어들을 정의합니다.
    """
    super(MLP, self).__init__() # 부모 클래스인 nn.Module의 초기화 메서드 호출

    # 첫 번째 완전 연결 계층 (입력 크기: 28x28, 출력 크기: 512)
    self.fc1 = nn.Linear(28*28, 512)

    # 두 번째 완전 연결 계층 (입력 크기: 512, 출력 크기: 10)
    self.fc2 = nn.Linear(512, 10)

  def forward(self, x):
    """
    순전파 메서드.
    - x: 입력 데이터 (배치 크기, 28, 28 크기의 이미지 텐서)
    """

    # 이미지는 28x28 픽셀 크기이며, 이를 1차원 벡터로 변환
    x = x.view(-1, 28*28) # 배치 크기에 맞게 자동으로 크기 조정

    # 첫 번째 완전 연결 계층을 통과
    x = self.fc1(x)

    # ReLU 활성화 함수 적용 (비선형 변환)
    x = F.relu(x)

    # 두 번째 완전 연결 계층 통과
    x = self.fc2(x)

    # 소프트맥스 함수 적용하여 클래스 확률을 계산
    return F.softmax(x, dim =1) # 각 클래스에 대한 확률 분포 계산

In [3]:
# MLP 모델 인스턴스 생성
model = MLP()
# 손실 함수로 CrossEntropyLoss 사용
criterion = nn.CrossEntropyLoss()
# 옵티마이저로 SGD 사용
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [4]:
def train(model, device, train_loader, optimizer, epoch):
  # 모델을 훈련 모드로 설정합니다. 훈련 시에는 드롭아웃이나 배치 정규화 등이 활성화됩니다.
  model.train()

  # train_loader에서 배치 단위로 데이터를 가져와서 학습을 진행합니다.
  for batch_idx, (data, target) in enumerate(train_loader):
    # 데이터를 지정된 장치(CPU 또는 GPU)로 이동
    data, target = data.to(device), target.to(device)

    # 이전에 계산된 그래디언트를 초기화
    optimizer.zero_grad()

    # 모델을 사용하여 입력 데이터를 처리
    output = model(data)

    # 모델 출력과 실제 레이블(target) 간의 손실(loss) 계산
    loss = criterion(output, target)

    # 손실에 따라 그래디언트 계산 (역전파 수행)
    loss.backward()

    # 옵티마이저를 통해 모델의 파라미터 업데이트
    optimizer.step()

    # 10번째 배치마다 학습 상태 출력
    if batch_idx % 10 == 0:
        print(f'Train Epoch: {epoch+1} '
              f'[{batch_idx * len(data)}/{len(train_loader.dataset)} '
              f'({100. * batch_idx / len(train_loader):.0f}%)]\t'
              f'Loss: {loss.item():.6f}')

In [5]:
def test(model, device, test_loader):
  # 모델을 평가 모드로 설정 (드롭아웃 및 배치 정규화 비활성화)
  model.eval()

  # 테스트 손실 누적 값 초기화
  test_loss = 0
  # 테스트 데이터 중 정답을 맞춘 개수를 세기 위한 변수 초기화
  correct = 0

  # 평가 모드에서는 그래디언트를 계산하지 않음
  with torch.no_grad():
    # 테스트 데이터 로더에서 배치를 하나씩 가져오기
    for data, target in test_loader:
      # 데이터를 지정된 장치(CPU 또는 GPU)로 이동
      data, target = data.to(device), target.to(device)

      # 모델을 사용하여 입력 데이터를 처리
      output = model(data)

      # 손실 값 계산 및 누적 (전체 손실을 계산하기 위해 추가)
      test_loss += criterion(output, target).item()

      # 모델의 출력에서 가장 높은 값을 가지는 클래스 예측
      pred = output.argmax(dim=1, keepdim=True)

      # 예측값과 실제값을 비교하여 일치하는 경우 정답 개수 증가
      correct += pred.eq(target.view_as(pred)).sum()

  # 테스트 데이터셋 크기로 나누어 평균 손실 계산
  test_loss /= len(test_loader.dataset)
  # 테스트 결과 출력: 평균 손실과 정확도
  print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: '
          f'{correct}/{len(test_loader.dataset)} '
          f'({100. * correct / len(test_loader.dataset):.0f}%)\n', end='\r')

In [6]:
n_epoch = 5 # 학습할 총 에폭 수를 설정

# CUDA(GPU)가 사용 가능하면 'cuda' 장치를 사용 그렇지 않으면 'cpu' 사용
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 모델을 지정된 장치(GPU 또는 CPU)로 이동
model.to(device)

for epoch in range(n_epoch):
  train(model, device, train_loader, optimizer, epoch)
  test(model, device, test_loader)

# 모델의 상태를 저장
torch.save(model.state_dict(), 'models/mnist_mpl_model_{}.pth'.format(n_epoch))
# 모델의 가중치를 models/mnist_mpl_model_{}.pth 파일에 저장
# 파일명에는 학습한 에폭 수를 포함시켜 추적 가능하도록 설정

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.292783
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.294613
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.289229
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.288550
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.283856
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.264317
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.266887
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.243804
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.233128
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.199783
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.211472
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.176999
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.123552
Train Epoch: 1 [8320/60000 (14%)]	Loss: 2.111541
Train Epoch: 1 [8960/60000 (15%)]	Loss: 2.128970
Train Epoch: 1 [9600/60000 (16%)]	Loss: 2.068211
Train Epoch: 1 [10240/60000 (17%)]	Loss: 2.010757
Train Epoch: 1 [10880/60000 (18%)]	Loss: 2.049391
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.968267
Train Epoch: 1 [12160/60000 (20%)]	Loss: 1.976975
Train Epoch: 1 [12800/60000 (

RuntimeError: Parent directory models does not exist.